In [10]:
from pathlib import Path
import os
import pprint
import pandas as pd

def series_equal(series_a, series_b):
    if len(series_a) != len(series_b):
        return False
    for element_a, element_b in zip(series_a, series_b):
        if element_a != element_b:
            return False
    return True

def get_fitness_csv_dirs(root_dir: Path):
    fitness_csv_dirs = []
    for root, _, files in os.walk(root_dir.expanduser()):
        if 'fitness.csv' in files and Path(root)/'fitness.csv' not in fitness_csv_dirs:
            fitness_csv_dirs.append(Path(root)/'fitness.csv')
    fitness_csv_dirs.sort()
    return fitness_csv_dirs

def filter_for_folder(dirs, folder_name):
    new_dirs = []
    for dir_ in dirs:
        if folder_name in dir_.parts:
            new_dirs.append(dir_)
    return new_dirs

def filter_for_multiple_folders(dirs, folder_names):
    if len(folder_names) == 0:
        return dirs
    else:
        return filter_for_multiple_folders(filter_for_folder(dirs, folder_names[0]), folder_names[1:])

class TeamFit():
    def __init__(self, rover, uav, team):
        self.rover = rover
        self.uav = uav
        self.team = team

def check_fitness_df(fitness_df):
    """Return true if agent fitnesses match team fitness. Return false if not"""
    # Look through the header to identify how many teams are in here
    names = fitness_df.columns
    num_teams = int(names[-3][-1]) # and the aggregated (we'll put that first)

    # Clump up each team into it's own object including rover's fitness, uav fitness, and team fitness
    # (Although technically, there is no temporal relationship between team n's fitness from one generation
    # to the next. The numbering is just for convenience, but each team is completely distinct from the team
    # generated at the previous generation with, even though they share the same id number)
    team_fits = [
        TeamFit(
            rover = fitness_df['rover_0_'],
            uav = fitness_df['uav_0'],
            team = fitness_df['team_fitness_aggregated']
        )
    ] + [
        TeamFit(
            rover = fitness_df['team_'+str(i)+'_rover_0'],
            uav = fitness_df['team_'+str(i)+'_uav_0'],
            team = fitness_df['team_fitness_'+str(i)]
        )
        for i in range(num_teams)
    ]

    # Now check that each team fit has a matching value for each rover uav and team fit
    for team_fit in team_fits:
        if not series_equal(team_fit.team, team_fit.rover) or not series_equal(team_fit.team, team_fit.uav):
            return False
    return True

def check_multiple_fitness_dfs(fitness_dfs):
    """Return True if every fitness csv checked contains no errors in fitness. False if there are errors"""
    for fitness_df in fitness_dfs:
        if not check_fitness_df(fitness_df):
            return False
    return True

# Comb through all of the results in quartz
fitness_csv_dirs = get_fitness_csv_dirs(root_dir=Path('~/influence-shaping/results/10_29_2024/quartz'))

# pprint.pprint(fitness_csv_dirs)

# Focus in on the 1_rover_1_uav/random_pois_10x10 trials using D-Indirect and G
dirs_of_interest = filter_for_multiple_folders(fitness_csv_dirs, ['1_rover_1_uav', 'random_pois_10x10', 'IndirectDifferenceAutomatic']) + \
    filter_for_multiple_folders(fitness_csv_dirs, ['1_rover_1_uav', 'random_pois_10x10', 'Global'])

# pprint.pprint(dirs_of_interest)

# Get each fitness.csv file
fitness_dfs = [pd.read_csv(dir_) for dir_ in dirs_of_interest]

# Now check all of them
check_multiple_fitness_dfs(fitness_dfs)


True

In [ ]:
# Sanity check. This should return False because D is not equal to G for uavs
difference_dirs = filter_for_multiple_folders(fitness_csv_dirs, ['1_rover_1_uav', 'random_pois_10x10', 'Difference'])
difference_fitness_dirs = [pd.read_csv(dir_) for dir_ in difference_dirs]
check_multiple_fitness_dfs(difference_fitness_dirs)

False